Pipeline:
    0. preparation: import statements, paths, inspect images
    1. transform images with transparent background
    2. move images to start of pipeline
    3. make images binary colored
    4. make all backgrounds black and figure white
    4a. just to experiment: skeletonize images
    5. trim borders
    6. resize images to 10x10 pixel
    7. flatten and write result to csv

# import statements

In [100]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import skimage

from skimage import io, exposure, img_as_float
from skimage.transform import resize
from skimage.util import invert
from skimage.morphology import skeletonize
from skimage.filters import threshold_minimum
from PIL import Image, ImageChops

# paths

In [101]:
# define all paths

projectpath = 'projectpath'

# training data
#srcpath = projectpath+'/01_data/Bilder/train'

# test data
srcpath = projectpath+'/01_data/Bilder/test'

startpath = projectpath+'/02_start/'

binarypath = projectpath+'/03_binary/'

whiteonblackpath = projectpath+'/04_white_on_black/'

skeletonpath = projectpath+'/04a_skeletonized/'

trimmedpath = projectpath+'/05_trimmed/'

resizedpath = projectpath+'/06_resized/'

resultpath = projectpath+'/07_result/'

# inspect images

In [102]:
# inspect images: dtype and shape

i8 = 0
i16 = 0
iother = 0

for pic in os.listdir(srcpath):
    image = io.imread(srcpath+'/'+pic)
    if(image.dtype == 'uint8'):
        i8 = i8+1
    elif(image.dtype == 'uint16'):
        i16 = i16+1
    else:
        ielse = iother+1
    
print("uint8: ", i8)
print("uint16: ", i16)
print("others: ", iother)

print("\nshapes:")
for pic in os.listdir(srcpath):
    image = io.imread(srcpath+'/'+pic)
    #print(image.shape)

uint8:  380
uint16:  335
others:  0

shapes:


# transform images (png) with transparent background

In [103]:
# function to transform png with transparent background to normal images
# series u013, u017, u031 and u032 are affected
# taken from https://www.linkedin.com/pulse/afternoon-debugging-e-commerce-image-processing-nikhil-rasiwasia/

def read_transparent_png(filename):
    image_4channel = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    alpha_channel = image_4channel[:,:,3]
    rgb_channels = image_4channel[:,:,:3]
    
    # White Background Image
    white_background_image = np.ones_like(rgb_channels, dtype=np.uint8) * 255
    
    # Alpha factor
    alpha_factor = alpha_channel[:,:,np.newaxis].astype(np.float32) / 255.0
    alpha_factor = np.concatenate((alpha_factor,alpha_factor,alpha_factor), axis=2)

    # Transparent Image Rendered on White Background
    base = rgb_channels.astype(np.float32) * alpha_factor
    white = white_background_image.astype(np.float32) * (1 - alpha_factor)
    final_image = base + white
    return final_image.astype(np.uint8)

In [104]:
for pic in os.listdir(srcpath):
    image = io.imread(srcpath+'/'+pic)
    try:
        image = read_transparent_png(srcpath+'/'+pic)
        cv2.imwrite(srcpath+'/'+pic, image)
    except:
        pass

# move images into start location

In [105]:
# move images into start location

for pic in os.listdir(srcpath):
    image = cv2.imread(srcpath+'/'+pic, cv2.IMREAD_UNCHANGED)
    # make all images uint8 type first - doesn't work properly for gray images
    #image = image.astype('uint8')
    io.imsave(startpath+pic, image)

# make images binary

In [106]:
# make images binary

for pic in os.listdir(startpath):
    image_gray = cv2.imread(startpath+pic, cv2.IMREAD_GRAYSCALE)
    (thresh, image_binary) = cv2.threshold(image_gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    cv2.imwrite(binarypath+pic, image_binary)

# count pixels and take greater value for background, invert image if background is white

In [107]:
# Make all images have a black background and a white figure.
# If number of non zero pixels (white) is greater than the number of zero pixels (black),
# the image is assumed to have a white background, therefore invert image.
# 255 means white, 0 means black.

for pic in os.listdir(binarypath):
    image = cv2.imread(binarypath+pic, cv2.IMREAD_UNCHANGED)
    total = image.shape[0]*image.shape[1]
    if(cv2.countNonZero(image) > (total - cv2.countNonZero(image))):
        image_inverted = cv2.bitwise_not(image)
        cv2.imwrite(whiteonblackpath+pic, image_inverted)
    else:
        cv2.imwrite(whiteonblackpath+pic, image)

# skeletonize

In [108]:
# skeltonize images

for pic in os.listdir(whiteonblackpath):
    image = io.imread(whiteonblackpath+pic)
    image = image / 255
    image_skeleton=skeletonize(image)
    image_skeleton = image_skeleton*255
    cv2.imwrite(skeletonpath+pic, image_skeleton)

# trim borders

In [109]:
# trim borders of images
# taken from https://stackoverflow.com/questions/10615901/trim-whitespace-using-pil/10616717#10616717

def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)

for pic in os.listdir(skeletonpath):
    image = Image.open(skeletonpath+pic)
    image = trim(image)
    image.save(trimmedpath+pic)

# resize image to 10x10 pixel size

In [110]:
# resize all images to 10x10 pixel images
# define size globally

dim = (10,10)
count = 10*10

for pic in os.listdir(trimmedpath):
    image = io.imread(trimmedpath+pic)
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    cv2.imwrite(resizedpath+pic, resized)

# flatten images and save

In [111]:
# flatten image values and save to csv file
count = 10*10

firstline = 'img_name'
for i in range(count):
    if(len(str(i)) == 1):
        firstline = firstline+',x00'+str(i)
    elif(len(str(i)) == 2):
        firstline = firstline+',x0'+str(i)
    elif(len(str(i)) == 3):
        firstline = firstline+',x'+str(i)
        
with open(projectpath+"/result.csv", 'a') as f:
        f.write(firstline+'\n')
        
for pic in os.listdir(resizedpath):
    image = plt.imread(resizedpath+pic)
    #thresh = threshold_minimum(image_resized)
    thresh = 0.0000001
    binary = image < thresh
    binary = binary.astype(int)
    plt.imsave(resultpath+pic,binary,cmap="binary")
    
    flat = binary.flatten()
    with open(projectpath+"/result.csv", 'a') as f:
        flat_str = ','.join(map(str,flat))
        line = pic+','+flat_str+'\n'
        f.write(line)
